In [20]:
import pymongo
import pandas
import numpy as np
from sklearn.neighbors import NearestNeighbors
# 设置精度
np.set_printoptions(precision=3)

In [ ]:
# mongodb uri
conn_str = "mongodb://1.15.118.125:27017/"
# 客户端连接
client = pymongo.MongoClient(conn_str, serverSelectionTimeoutMS=5000)
collection_user = client.get_database("Lab").get_collection("Mission")

In [2]:
# 定义函数用

In [3]:
# 查询 title 与对应的 tags
title_with_tags = {}
for doc in collection_user.find():
    mission_tags = doc.get("missionTags")
    mission_title = doc.get("title")
    title_with_tags[mission_title] = mission_tags

# pands 格式化输出
ser_tag_tags = pandas.Series(title_with_tags)
ser_tag_tags.head(len(ser_tag_tags))

选调知多少政策宣讲活动                                                                                             [就业发展, 宣讲]
会计专业课程组本科教学汇报                                                                                        [教学建设, 报告/汇报]
喻园管理论坛: 重大工程中的使命导向型创新--以都江堰系统工程为例                                                                         [学术, 讲座]
国奖专访                                                                                                    [采访, 校园生活]
AOL第三轮工作会议                                                                                              [教学建设, 会议]
校运会跟拍                                                                                                   [文体活动, 赛事]
学习贯彻党的二十大精神宣讲报告会                                                                                 [党团活动, 宣讲, 报告/汇报]
“廖建桥教室”揭牌仪式活动                                                                                           [教学建设, 仪式]
喻园管理论坛: 数智时代数据应用与组织效能研究                                                         

In [4]:
# 查询 tag 的频次
def get_tag_times(dict_value_tags):
    tag_with_time = {}
    for key in dict_value_tags:
        for tag in dict_value_tags[key]:
            if tag not in tag_with_time:
                tag_with_time[tag] = 1
            else:
                tag_with_time[tag] += 1
    return tag_with_time


# pands 格式化输出
ser_tag_with_time = pandas.Series(get_tag_times(title_with_tags))
ser_tag_with_time.head(len(ser_tag_with_time))

就业发展     1
宣讲       3
教学建设     6
报告/汇报    4
学术       7
讲座       6
采访       1
校园生活     4
会议       3
文体活动     2
赛事       2
党团活动     3
仪式       3
趣味活动     1
师生交流     2
学生工作     1
dtype: int64

In [ ]:
# 定义 0 矩阵
dist = np.zeros((len(title_with_tags), len(get_tag_times(title_with_tags)),))
# 获取 tags 列表
tags_list = list(get_tag_times(title_with_tags).keys())
print(tags_list)
for mission_index in range(len(title_with_tags)):
    for tag in title_with_tags[mission_index]:
        for tag_index in range(len(tags_list)):
            if tag == tags_list[tag_index]:
                dist[mission_index][tag_index] = 1

In [37]:
# 使用 KNN 算法计算样本相互距离
calcu_nbr = NearestNeighbors(n_neighbors=10, algorithm='ball_tree').fit(dist)
distances,indices = calcu_nbr.kneighbors(dist)

[[0.    1.414 1.732 1.732 2.    2.    2.    2.    2.    2.   ]
 [0.    1.    1.414 1.414 1.414 1.414 1.414 1.732 1.732 2.   ]
 [0.    0.    0.    0.    0.    1.414 1.414 1.732 1.732 2.   ]
 [0.    1.414 1.414 1.414 1.732 2.    2.    2.    2.    2.   ]
 [0.    0.    0.    1.    1.414 1.414 2.    2.    2.    2.   ]
 [0.    0.    1.732 2.    2.    2.    2.    2.    2.    2.   ]
 [0.    1.    1.732 1.732 1.732 1.732 2.    2.    2.236 2.236]
 [0.    1.    1.414 1.414 1.414 1.414 1.414 1.732 2.    2.   ]
 [0.    0.    0.    0.    0.    1.414 1.414 1.732 1.732 2.   ]
 [0.    0.    0.    1.    1.414 1.414 2.    2.    2.    2.   ]
 [0.    0.    1.732 2.    2.    2.    2.    2.    2.    2.   ]
 [0.    1.414 1.414 1.414 1.732 2.    2.    2.    2.    2.   ]
 [0.    0.    0.    0.    0.    1.414 1.414 1.732 1.732 2.   ]
 [0.    0.    0.    0.    0.    1.414 1.414 1.732 1.732 2.   ]
 [0.    0.    0.    0.    0.    1.414 1.414 1.732 1.732 2.   ]
 [0.    1.414 1.414 1.414 1.414 1.414 1.414 1.732 1.732

In [39]:
# 输出距离
print(distances)

[[0.    1.414 1.732 1.732 2.    2.    2.    2.    2.    2.   ]
 [0.    1.    1.414 1.414 1.414 1.414 1.414 1.732 1.732 2.   ]
 [0.    0.    0.    0.    0.    1.414 1.414 1.732 1.732 2.   ]
 [0.    1.414 1.414 1.414 1.732 2.    2.    2.    2.    2.   ]
 [0.    0.    0.    1.    1.414 1.414 2.    2.    2.    2.   ]
 [0.    0.    1.732 2.    2.    2.    2.    2.    2.    2.   ]
 [0.    1.    1.732 1.732 1.732 1.732 2.    2.    2.236 2.236]
 [0.    1.    1.414 1.414 1.414 1.414 1.414 1.732 2.    2.   ]
 [0.    0.    0.    0.    0.    1.414 1.414 1.732 1.732 2.   ]
 [0.    0.    0.    1.    1.414 1.414 2.    2.    2.    2.   ]
 [0.    0.    1.732 2.    2.    2.    2.    2.    2.    2.   ]
 [0.    1.414 1.414 1.414 1.732 2.    2.    2.    2.    2.   ]
 [0.    0.    0.    0.    0.    1.414 1.414 1.732 1.732 2.   ]
 [0.    0.    0.    0.    0.    1.414 1.414 1.732 1.732 2.   ]
 [0.    0.    0.    0.    0.    1.414 1.414 1.732 1.732 2.   ]
 [0.    1.414 1.414 1.414 1.414 1.414 1.414 1.732 1.732

In [40]:
# 输出邻近的 index 值
print(indices)

[[ 0 19  6 20  2  4  3  8  1  7]
 [ 1 20  4  9  7 17 18  6 23  8]
 [ 8 12 13  2 14 17 15 20 23  7]
 [ 3 16 22 11 20  1  8  4  9  7]
 [ 9  4 18 20  1  7  0  3  2  8]
 [ 5 10 20  3  0  4  8  1  9  7]
 [ 6 19  1 17  0 15 23 20  8  7]
 [ 7 20  9  4 18 21  1 23  8  3]
 [ 8 12 13  2 14 17 15 20 23  7]
 [ 9  4 18 20  1  7  0  3  2  8]
 [ 5 10 20  3  0  4  8  1  9  7]
 [11 16 22  3 20  1  8  4  9  7]
 [ 8 12 13  2 14 17 15 20 23  7]
 [ 8 12 13  2 14 17 15 20 23  7]
 [ 8 12 13  2 14 17 15 20 23  7]
 [15 12 19 13  8  2 14  6 20  7]
 [16 22 11  3 20  1  8  4  9  7]
 [17 23 13  8  2 12  1 14 20  6]
 [ 9  4 18 20  1  7  0  3  2  8]
 [19  6 15  0 20  3  1  8  4  7]
 [20  9  1 18  4  7  8  3  2  0]
 [21  7 23 20  3  4  8  9  0  1]
 [16 22 11  3 20  1  8  4  9  7]
 [23 17 13 21  1 14  8 12  2  7]]


In [56]:
# 获取 title 的 list
title_list = list(title_with_tags.keys())
print(title_list)
# 根据矩阵进行寻找
def find_similar_mission(title_str,num):
    similar_mission_list = []
    my_index = title_list.index(title_str)
    print("为 " + title_str+" 寻找相似任务")
    pointer = 0
    count = 0
    while count < num:
        if indices[my_index][pointer] == my_index:
            pointer += 1
            continue
        else:
            similar_mission_list.append(title_list[indices[my_index][pointer]])
            count += 1
            pointer += 1
    print(similar_mission_list)
    return similar_mission_list

['选调知多少政策宣讲活动', '会计专业课程组本科教学汇报', '喻园管理论坛: 重大工程中的使命导向型创新--以都江堰系统工程为例', '国奖专访', 'AOL第三轮工作会议', '校运会跟拍', '学习贯彻党的二十大精神宣讲报告会', '“廖建桥教室”揭牌仪式活动', '喻园管理论坛: 数智时代数据应用与组织效能研究', '领学团第一次集体备课会', '华工杯乒乓球赛半决赛', '热缩片DIY活动', '喻园管理论坛: Intelligent Simulation Optimization: An Example in Multi-fidelity Simulation Modeling', '喻园管理论坛: Auditor Industry Range and Professional Skepticism', '喻园管理论坛: How Banks Understand R&D Investments: The Case of Standards Setting Organizations', '发展对象开班, 杨院长教授党课', '十一月舍导交流', '管理学院第二届工商管理学科高端论坛系列活动: 学术报告会', '管理学院第二届工商管理学科高端论坛系列活动: 学科建设座谈会', '二十大宣讲', '新进老师培训', '学生骨干训练营的结营仪式', '十二月舍导交流', '中国管理学前沿研究研讨会: 开幕式与主报告']


In [57]:
find_similar_mission('喻园管理论坛: 重大工程中的使命导向型创新--以都江堰系统工程为例',3)
find_similar_mission('二十大宣讲',3)

为 喻园管理论坛: 重大工程中的使命导向型创新--以都江堰系统工程为例 寻找相似任务
['喻园管理论坛: 数智时代数据应用与组织效能研究', '喻园管理论坛: Intelligent Simulation Optimization: An Example in Multi-fidelity Simulation Modeling', '喻园管理论坛: Auditor Industry Range and Professional Skepticism']
为 二十大宣讲 寻找相似任务
['学习贯彻党的二十大精神宣讲报告会', '发展对象开班, 杨院长教授党课', '选调知多少政策宣讲活动']


['学习贯彻党的二十大精神宣讲报告会', '发展对象开班, 杨院长教授党课', '选调知多少政策宣讲活动']